# Simulations

- hide: true
- toc: true
- badges: true
- comments: true
- categories: [stats]

In [1]:
import numpy as np

## Simulating loan repayments using Monte Carlo simulation

Entirely based on [this](https://datascience.quantecon.org/scientific/randomness.html) excellent QuantEcon tutorial.

A small company makes loans of up to 25k to be repaid a year later. 75 percent of loans are repaid in full, 20 percent of loans are repaid in half, the rest are not repaid. The company discounts the future at an annual rate of 5 percent. How much would the company be willing to loan if it wants to break even on average?

We can calculate this by hand. One year from now, the average repayment is 0.75(25000) + 0.2(12500) + 0.05(0) = 21250. Because that repayment occurs a year from now while the loan is made today, the company calculates the repayment's present value as: (1/1.05)21250 = 20238.1. Assuming that loan repayment is independent of loan size, then this is the loan size for which the company will break even on average (if it makes enough loans).

We can verify this using a simulation:

In [110]:
def repayment_simulator(n, r=0.05, full_repayment=25_000, partial_repayment=12_500):
    """Simulate present value of repayments for N loans."""
    repayments = np.zeros(n)
    outcome = np.random.rand(n)
    
    full_repayments = outcome <= 0.75
    repayments[full_repayments] = full_repayment
    partial_repayments = (outcome <= 0.95) & ~full_repayments
    repayments[partial_repayments] = partial_repayment
    
    return (1 / (1 + r)) * repayments

np.mean(repayment_simulator(10_000_000))

20237.311904761867

Instead of breaking even, the company is now looking for the largest loan size that will give it a 95 percent chance of being profitable in a year it makes 250 loans.

In [120]:
def simulate_year(n, k):
    """Simulate a year with N loans k times."""
    year_avgs = np.zeros(k)
    for year in range(k):
        year_avgs[year] = np.mean(repayment_simulator(n))
    
    return year_avgs
    
np.percentile(simulate_year(250, 10_000), [5, 50, 95])

array([19523.80952381, 20238.0952381 , 20904.76190476])

The result tells us that in 95 percent of simulated years, the average repayment is larger than 19,523. Hence, if the company wants to make a profit with 95 percent probability, it should loan no more than this amount.

## Sources

- [QuantEcon](https://datascience.quantecon.org/scientific/randomness.html)